In [26]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import random
from datetime import datetime
# Import the time module.
import time
# Get today's date in seconds.
today = time.time()
# Import the API key.
from config import weather_api_key

In [27]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90., high=90., size=2000)
lngs = np.random.uniform(low=-180., high=180., size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [28]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [29]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [30]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

760

In [31]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=14d496981f3ab4ab4fbc15adae63ab96


In [32]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]("{:.0f}".format)
        city_clouds = city_weather["clouds"]["all"]("{:.0f}".format)
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        #city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description,})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | dingle
City not found. Skipping...
Processing Record 2 of Set 1 | mataura
City not found. Skipping...
Processing Record 3 of Set 1 | rikitea
City not found. Skipping...
Processing Record 4 of Set 1 | saleaula
City not found. Skipping...
Processing Record 5 of Set 1 | hermanus
City not found. Skipping...
Processing Record 6 of Set 1 | ponta do sol
City not found. Skipping...
Processing Record 7 of Set 1 | ushuaia
City not found. Skipping...
Processing Record 8 of Set 1 | muzhi
City not found. Skipping...
Processing Record 9 of Set 1 | zaranj
City not found. Skipping...
Processing Record 10 of Set 1 | clyde river
City not found. Skipping...
Processing Record 11 of Set 1 | punta arenas
City not found. Skipping...
Processing Record 12 of Set 1 | yaan
City not found. Skipping...
Processing Record 13 of Set 1 | yanchukan
City not found. Skipping...
Processing Record 14 of Set 1 | hambantota
City not fo

City not found. Skipping...
Processing Record 19 of Set 3 | rawson
City not found. Skipping...
Processing Record 20 of Set 3 | monrovia
City not found. Skipping...
Processing Record 21 of Set 3 | ossora
City not found. Skipping...
Processing Record 22 of Set 3 | georgetown
City not found. Skipping...
Processing Record 23 of Set 3 | butaritari
City not found. Skipping...
Processing Record 24 of Set 3 | padang
City not found. Skipping...
Processing Record 25 of Set 3 | tsihombe
City not found. Skipping...
Processing Record 26 of Set 3 | alipur
City not found. Skipping...
Processing Record 27 of Set 3 | aswan
City not found. Skipping...
Processing Record 28 of Set 3 | mount isa
City not found. Skipping...
Processing Record 29 of Set 3 | umm lajj
City not found. Skipping...
Processing Record 30 of Set 3 | sola
City not found. Skipping...
Processing Record 31 of Set 3 | norrtalje
City not found. Skipping...
Processing Record 32 of Set 3 | fengrun
City not found. Skipping...
Processing Recor

City not found. Skipping...
Processing Record 39 of Set 5 | mapiripan
City not found. Skipping...
Processing Record 40 of Set 5 | vardo
City not found. Skipping...
Processing Record 41 of Set 5 | torbay
City not found. Skipping...
Processing Record 42 of Set 5 | carroll
City not found. Skipping...
Processing Record 43 of Set 5 | henties bay
City not found. Skipping...
Processing Record 44 of Set 5 | neuquen
City not found. Skipping...
Processing Record 45 of Set 5 | larsnes
City not found. Skipping...
Processing Record 46 of Set 5 | champerico
City not found. Skipping...
Processing Record 47 of Set 5 | pangnirtung
City not found. Skipping...
Processing Record 48 of Set 5 | sistranda
City not found. Skipping...
Processing Record 49 of Set 5 | vaitupu
City not found. Skipping...
Processing Record 50 of Set 5 | yelizovo
City not found. Skipping...
Processing Record 1 of Set 6 | toftir
City not found. Skipping...
Processing Record 2 of Set 6 | languyan
City not found. Skipping...
Processin

City not found. Skipping...
Processing Record 8 of Set 8 | san quintin
City not found. Skipping...
Processing Record 9 of Set 8 | omboue
City not found. Skipping...
Processing Record 10 of Set 8 | arroyo
City not found. Skipping...
Processing Record 11 of Set 8 | krasnoyarsk-45
City not found. Skipping...
Processing Record 12 of Set 8 | elko
City not found. Skipping...
Processing Record 13 of Set 8 | temaraia
City not found. Skipping...
Processing Record 14 of Set 8 | colesberg
City not found. Skipping...
Processing Record 15 of Set 8 | fairbanks
City not found. Skipping...
Processing Record 16 of Set 8 | litoral del san juan
City not found. Skipping...
Processing Record 17 of Set 8 | westport
City not found. Skipping...
Processing Record 18 of Set 8 | hamilton
City not found. Skipping...
Processing Record 19 of Set 8 | fortuna
City not found. Skipping...
Processing Record 20 of Set 8 | juneau
City not found. Skipping...
Processing Record 21 of Set 8 | taber
City not found. Skipping...

City not found. Skipping...
Processing Record 26 of Set 10 | kutum
City not found. Skipping...
Processing Record 27 of Set 10 | manicore
City not found. Skipping...
Processing Record 28 of Set 10 | codrington
City not found. Skipping...
Processing Record 29 of Set 10 | palabuhanratu
City not found. Skipping...
Processing Record 30 of Set 10 | isangel
City not found. Skipping...
Processing Record 31 of Set 10 | linhares
City not found. Skipping...
Processing Record 32 of Set 10 | klyuchi
City not found. Skipping...
Processing Record 33 of Set 10 | ust-kamchatsk
City not found. Skipping...
Processing Record 34 of Set 10 | churapcha
City not found. Skipping...
Processing Record 35 of Set 10 | rudnogorsk
City not found. Skipping...
Processing Record 36 of Set 10 | nizhnyaya pavlovka
City not found. Skipping...
Processing Record 37 of Set 10 | brigantine
City not found. Skipping...
Processing Record 38 of Set 10 | ibicui
City not found. Skipping...
Processing Record 39 of Set 10 | valavanur

City not found. Skipping...
Processing Record 43 of Set 12 | bargal
City not found. Skipping...
Processing Record 44 of Set 12 | zhezkazgan
City not found. Skipping...
Processing Record 45 of Set 12 | waltershausen
City not found. Skipping...
Processing Record 46 of Set 12 | spearfish
City not found. Skipping...
Processing Record 47 of Set 12 | hashtrud
City not found. Skipping...
Processing Record 48 of Set 12 | opelousas
City not found. Skipping...
Processing Record 49 of Set 12 | rio grande
City not found. Skipping...
Processing Record 50 of Set 12 | iskateley
City not found. Skipping...
Processing Record 1 of Set 13 | antofagasta
City not found. Skipping...
Processing Record 2 of Set 13 | krasnoselkup
City not found. Skipping...
Processing Record 3 of Set 13 | grindsted
City not found. Skipping...
Processing Record 4 of Set 13 | inta
City not found. Skipping...
Processing Record 5 of Set 13 | gizo
City not found. Skipping...
Processing Record 6 of Set 13 | ngunguru
City not found. 

City not found. Skipping...
Processing Record 11 of Set 15 | dianopolis
City not found. Skipping...
Processing Record 12 of Set 15 | hirara
City not found. Skipping...
Processing Record 13 of Set 15 | gallup
City not found. Skipping...
Processing Record 14 of Set 15 | rabo de peixe
City not found. Skipping...
Processing Record 15 of Set 15 | veseloyarsk
City not found. Skipping...
Processing Record 16 of Set 15 | winton
City not found. Skipping...
Processing Record 17 of Set 15 | staryy nadym
City not found. Skipping...
Processing Record 18 of Set 15 | sabang
City not found. Skipping...
Processing Record 19 of Set 15 | xining
City not found. Skipping...
Processing Record 20 of Set 15 | valladolid
City not found. Skipping...
Processing Record 21 of Set 15 | salinas
City not found. Skipping...
Processing Record 22 of Set 15 | udachnyy
City not found. Skipping...
Processing Record 23 of Set 15 | vila
City not found. Skipping...
Processing Record 24 of Set 15 | solnechnyy
City not found. S

In [33]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_datad_df.head(10)

NameError: name 'city_datad_df' is not defined

In [ ]:
city_datad_df.to_csv('WeatherPy_Database.csv')

In [ ]:
city_datad_df.head(10)